In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [2]:
train = pd.read_csv('Train_f_inclusion.csv' )
test= pd.read_csv('Test_f_inclusion.csv')
kenya = train[train['country'] == 'Kenya']
kenya_test = test[test['country'] == 'Kenya']
Tanzania = train[train['country'] == 'Tanzania']
Tanzania_test = test[test['country'] == 'Tanzania']
Rwanda = train[train['country'] == 'Rwanda']
Rwanda_test = test[test['country'] == 'Rwanda']
Uganda= train[train['country'] == 'Uganda']
Uganda_test = test[test['country'] == 'Uganda']
countries_train = [test,train,kenya,Tanzania,Rwanda,Uganda]
countries_test = [test,kenya_test, Tanzania_test,Rwanda_test,Uganda_test]
kenya

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6063,Kenya,2018,uniqueid_6067,No,Rural,Yes,4,26,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent
6064,Kenya,2018,uniqueid_6068,No,Urban,No,5,17,Female,Child,Single/Never Married,Secondary education,Remittance Dependent
6065,Kenya,2018,uniqueid_6069,No,Rural,Yes,9,30,Female,Spouse,Married/Living together,No formal education,Farming and Fishing
6066,Kenya,2018,uniqueid_6070,No,Rural,Yes,3,52,Male,Head of Household,Married/Living together,Primary education,Self employed


In [3]:
def model_features(data):
    bank_acc = data[data['country'] == 'Kenya']['bank_account']
    bank_acc =  bank_acc.replace('Yes',1)
    bank_acc =  bank_acc.replace('No',0)
    data =data.drop(['country','uniqueid','bank_account','year'],axis=1)
    text_cols = data.select_dtypes(include = ['object']).columns
    for col in text_cols:
        col_dummies = pd.get_dummies(data[col])
        data = pd.concat([data,col_dummies],axis = 1)
        del data[col]
    data['bank_acc'] = bank_acc
    return data
def modelelling(countries_train,number,model):
    kenya_update = model_features(countries_train[number])
    reg = model.LinearRegression()
    faatures = kenya_update.columns
    features = kenya_update[faatures].drop(['bank_acc'],axis=1)
    reg.fit(features, kenya_update['bank_acc'])
    predictions_kenya_train = pd.Series(reg.predict(features))
    bank_acc =  countries_train[2]['bank_account'].replace('Yes',1)
    bank_acc =  bank_acc.replace('No',0)
    combined = pd.concat([predictions_kenya_train, bank_acc],axis = 1)
    return combined

In [4]:
kenya_update = model_features(kenya)
kenya_combined= modelelling(countries_train,2,linear_model)
kenya_combined.columns =['predict','actual']
kenya_combined

,predict,actual
0,0.344692,1
1,0.172802,0
2,0.545666,1
3,0.317295,0
4,-0.039845,0
...,...,...
6063,0.066891,0
6064,0.035686,0
6065,0.029186,0
6066,0.342236,0


In [5]:
predictions_kenya_train_2 = kenya_combined['predict']

pred= []
for i in predictions_kenya_train_2:
    if i > 0.7:
        predictions_kenya_train_2[i] = 1
    else:
        predictions_kenya_train_2[i] = 0
    pred.append(predictions_kenya_train_2[i])
pred_2 = pd.Series(pred)

<ipython-input-5-afeb850eebd3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_kenya_train_2[i] = 0


In [6]:
kenya_combined['clean_predict']=pred_2

In [7]:
kenya_combined

,predict,actual,clean_predict
0,0.344692,1,0.0
1,0.172802,0,0.0
2,0.545666,1,0.0
3,0.317295,0,0.0
4,-0.039845,0,0.0
...,...,...,...
6063,0.066891,0,0.0
6064,0.035686,0,0.0
6065,0.029186,0,0.0
6066,0.342236,0,0.0


In [8]:
mse = mean_squared_error(kenya_combined['actual'], kenya_combined['clean_predict'])
rmse = np.sqrt(mse)

In [9]:
mse, rmse

(0.20764667106130522, 0.45568264292301636)

In [10]:
kenya_combined[kenya_combined['actual'] == kenya_combined['clean_predict']].shape[0] /kenya_combined.shape[0]

0.7923533289386948